Keras (ja TensorFlow) jatkoa
======================
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/11/TensorFlowLogo.svg/1229px-TensorFlowLogo.svg.png" 
alt="TensorFlow" width="400"/>
![Keras](https://upload.wikimedia.org/wikipedia/commons/c/c9/Keras_Logo.jpg "Keras")

Asentaaksesi tarvittavat paketit omalla koneellasi harjoituksen suorittamista varten:
```
$ pip3 install scikit-learn tensorflow==2.0.0rc1 numpy matplotlib==2.2.2
```

## Tekstin luokittelua
Tämä harjoitus on muokattu käyttäen [tutoriaalia](https://www.tensorflow.org/tutorials/keras/basic_text_classification)



### Aineiston lataus
Tällä kertaa aineistona käytetään [imdb](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification)-aineistoa, 
joka sisältää sanallisia arvosteluja numeroiksi muutettuna ja luokkana on joko positiivinen tai negatiivinen arvostelu.

**Ongelma**: Voiko arvostelun sävyn (positiivinen/negatiivinen) päätellä arvostelun tekstistä?

Ladataan arvostelut siten, että sanaston suuruus on 10000, eli arvostelut sisältävät 10000 useiten käytettyä sanaa.

In [30]:
from tensorflow import keras
imdb = keras.datasets.imdb
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)

Tarkastele aluksi hieman aineistoa. 

In [31]:
# datan katselua
print("Aineistojen koko: opetus:{}, testi:{}".format(len(y_train), len(y_test)))
print("Ensimmäinen havainto", X_train[0])
print("Ensimmäisten pituuksia", [len(x) for x in X_train[:5]])

Kuten huomasitkin, arvostelu koostuukin sanojen sijaan numeroista. Imdb-aineisto sisältää kuitenkin sanaston,
jonka avulla on mahdollista nähdä mitä alkuperäinen arvostelu pitää sisällään.

In [32]:
# Sanasto
word_index = imdb.get_word_index()

Luodaan alle funktio, jonka avulla on mahdollisus kääntää arvostelu numeroista tekstiksi. 
Ensimäiset 4 numeroa on varattu sanastossa erikoistilainteisiin. 
* 0 on merkki, joka mallin tulee ignoroida
* 1 kuvaa arvostelun alkua
* 2 kuvaa tuntematonta sanaa
* 3 on varattu muille

Koska aineisto sisältää nyt vain 10000 yleisintä sanaa, merkkiä 2 näkyy niiden sanojen kohdalla, jotka eivät ole 
yleisimpien sanojen joukossa.

In [33]:
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNKNOWN>"] = 2
word_index["<UNUSED>"] = 3

reverse_index = {v: k for (k, v) in word_index.items()}

def translate(sample):
    return ' '.join([reverse_index.get(i, '<not found>') for i in sample])

In [34]:
print("Ensimmäinen havainto sanoina:\n", translate(X_train[0]))

### Aineiston esikäsittelyä
Koska osa arvosteluista on eripituisia, ne on muutettava samanpituisiksi ennen luokittelua.

Yksi tapa tehdä tämä on lisätä arvostelun alkuun tai loppuun niin monta ignoroitavaa merkkiä (`word_index["<PAD>"]`,
eli 0), että kaikkien arvostelujen pituudet ovat samat. Mallille kerrotaan myöhemmin, että sitä 
arvoa ei tule ottaa huomioon. 

Toinen vaihtoehto on jättää pisimpien arvostelujen viimeisiä sanoja pois ja rajata kaikkien arvosteluen pituus
lyhimmän mukaan.

Nämä kaksi tapaa voi myös yhdistää. Silloin lyhimpien loppuun lisätään ignoroitavia merkkejä tiettyyn määrään asti,
ja niistä arvosteluista, jotka ylittävät tämän määrän, voidaan jättää ylimenevät merkit pois. Tähän tarkoitukseen 
soveltuu mainiosti metodi [pad_sequences](https://keras.io/preprocessing/sequence/#pad_sequences) 

Muodosta uudet `X_train_padded` ja `X_test_padded` siten, että lisäät ignoroitavia merkkejä loppuun ja siten, että
maksimipituudeksi tulee 256.

In [36]:
X_train_padded = keras.preprocessing.sequence.pad_sequences(X_train,
                                            value=word_index["<PAD>"],
                                            padding='post',
                                            maxlen=256)

In [37]:
X_test_padded = keras.preprocessing.sequence.pad_sequences(X_test,
                                            value=word_index["<PAD>"],
                                            padding='post',
                                            maxlen=256)

print(X_train_padded[0])
print(translate(X_train_padded[0]))

## Mallin luominen
Jaa seuraavaksi opetusjoukko vielä erikseen opetus- ja validointijoukkouhin käyttäen 
[`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)-metodia 
siten, että opetusjoukon osuus on 80% ja validointijoukon osuus on 20% alkuperäisen opetusjoukon koosta. 

Validointijoukkoa käytetään tässä opetuksen aikaiseen seuraamiseen.

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train_padded, y_train, test_size=0.2, random_state=42)

Luo ja kokoa malli `create_model`-funktion avulla. Käytä ensimmäisen harjoituksen tapaan [Sequential](https://keras.io/getting-started/sequential-model-guide/)
-luokkaa.

Luo neuronitasot seuraavasti:
1. Ensimmäinen taso on [Embedding](https://keras.io/layers/embeddings/#embedding), jonka avulla on mahdollista luoda
   sulautettu moniulotteinen Dense-taso sanaston koon perusteella. Anna `input_dim` arvoksi sanaston koko ja `output_dim` 
   arvoksi haluamasi Dense-tason koko (esimerkiksi 16).
2. Toinen taso on [GlobalAveragePooling1D](https://keras.io/layers/pooling/#globalaveragepooling1d), joka 
3. Kolmanneksi tasoksi aseta [Dense](https://keras.io/layers/core/#Dense) ja sen kooksi haluamasi koko. 
   Aseta sen aktivointifunktioksi "relu"
4. Voit asettaa lisää Dense-tasoja halutessasi 
5. Viimeiseksi tasoksi aseta [Dense](https://keras.io/layers/core/#Dense). Koska luokkia on vain kaksi, voidaan 'softmax'-
   aktivointifunktion sijaan käyttää 'sigmoid'-funktiota, joka palauttaa (todennäköisyysarvoksikin tulkittavan) luvun
    0-1 väliltä. Tällöin tason kooksi asetetaan 1.

Mallin kokoamiseksi käytä optimointifunktiona 'adam'-funktiota ja häviöfunktiona 'binary_crossentropy'-funktiota, 
sillä kyseessä on kahden luokan luokitteluongelma ja viimeinen taso antaa todennäköisyysarvon. 

Käytä `"accuracy"`-metriikka opetuksen seuraamiseksi.

In [44]:
def create_model():
    model = keras.models.Sequential()
    model.add(keras.layers.Embedding(10000, 16))
    model.add(keras.layers.GlobalAveragePooling1D())
    model.add(keras.layers.Dense(16, activation="relu"))
    model.add(keras.layers.Dense(1, activation="sigmoid"))
    
    print(model.summary())
    
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])
    return model

## Opetaaminen
Opeta nyt luomasi malli käyttäen `validation_data`-parametrina validointijoukkoa `(X_val, y_val)`. Aseta `batch_size`-
ja `epoch`-parametrit haluamiksesi.

In [46]:
model = create_model()
history = model.fit(X_train,
                    y_train,
                    epochs=2,
                    batch_size=512,
                    validation_data=(X_val, y_val),
                    verbose=1)

## Evaluoiminen
Evaluoi nyt luomasi malli käyttäen 1000 ensimmäistä testijoukon havaintoa.

In [48]:
evaluation = model.evaluate(X_test[:1000], y_test[:1000], batch_size=100)
print("Testijoukon häviö: {:.4f} OA: {:.4f}".format(evaluation[0], evaluation[1]))


